In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import cv2 as cv
import os
from tensorflow import keras
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from keras.models import Sequential

from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical

train_dir = r'/kaggle/input/animals-detection-images-dataset/train/'
test_dir = r'/kaggle/input/animals-detection-images-dataset/test/'

labels = ['Butterfly',
       'Caterpillar','Chicken', 'Crab',
       'Deer', 'Duck', 'Eagle','Fish',
       'Frog', 'Giraffe','Goose', 'Horse',
       'Jellyfish', 'Ladybug', 'Lion',
       'Lizard','Monkey','Owl', 'Parrot',
       'Penguin',
       'Shark','Snail', 'Snake',
       'Sparrow', 'Spider','Squirrel',
       'Tiger', 'Tortoise','Whale']

label_len = len(labels)
print(label_len)

# Varijable za trening podatke
X = []
Y = []

# Učitavanje podataka
for label in labels:
    folderpath = os.path.join(train_dir, label)
    
    for file in os.listdir(folderpath):
        img_path = os.path.join(folderpath, file)
        img = cv.imread(img_path)
        
        if img is not None:
            img = cv.resize(img, (224, 224))
            X.append(np.array(img))
            Y.append(labels.index(label))

# Konverzija listi u NumPy matrice
X = np.array(X)
Y = np.array(Y)

print("Dimenzije trening podataka:")
print("X shape:", X.shape)
print("Y shape:", Y.shape)

# Varijable za validacijske podatke
X_valid = []
Y_valid = []
X_valid_path = []

for label in labels:
    folderpath = os.path.join(test_dir, label)
    
    for file in os.listdir(folderpath):
        img_path = os.path.join(folderpath, file)
        img = cv.imread(img_path)
        
        if img is not None:
            img = cv.resize(img, (224, 224))
            X_valid.append(np.array(img))
            X_valid_path.append(img_path)
            Y_valid.append(labels.index(label))
            
X_valid = np.array(X_valid)
Y_valid = np.array(Y_valid)

print("\nDimenzije validacijskih podataka:")
print("X_valid shape:", X_valid.shape)
print("Y_valid shape:", Y_valid.shape)

total_images = X.shape[0] + X_valid.shape[0]
print("Broj slika:", total_images)

for i in range(3):
    plt.imshow(X[i])
    plt.title(f"Classe : {Y[i]}")
    plt.show()
    
base_model = ResNet50(weights=None, include_top=False, input_shape=(224, 224, 3))
weights_path = '/kaggle/input/tezine/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
base_model.load_weights(weights_path)

base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(len(labels), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Konverzija oznaka u kategorije one-hot
Y_one_hot = to_categorical(Y)
Y_valid_one_hot = to_categorical(Y_valid)

history = model.fit(X, Y_one_hot, epochs=20, validation_data=(X_valid, Y_valid_one_hot))

Y_pred = model.predict(X_valid)
Y_pred_classes = np.argmax(Y_pred, axis=1) 

conf_matrix = confusion_matrix(Y_valid, Y_pred_classes)
acc_score = accuracy_score(Y_valid, Y_pred_classes)
print("Tačnost rezultata:", acc_score)

plt.figure(figsize=(20,20))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)
plt.title('Matrica konfuzije')
plt.xlabel('Predikcije')
plt.ylabel('Stvarne vrednosti')
plt.show()

# Provera gubitaka
plt.plot(history.history['loss'], label='Gubitak na trening skupu')
plt.plot(history.history['val_loss'], label='Gubitak pri validaciji')
plt.title('Trening i Validacijski Gubitak-Istorija')
plt.ylabel('Gubitak')
plt.xlabel('Epohe')
plt.legend()
plt.show()

# Tačnost
plt.plot(history.history['accuracy'], label='Tačnost na trening skupu')
plt.plot(history.history['val_accuracy'], label='Tačnost pri validaciji')
plt.title('Trening i Validacijska Tačnost-Istorija')
plt.ylabel('Tačnost')
plt.xlabel('Epohe')
plt.legend()
plt.show()

report = classification_report(Y_valid, Y_pred_classes, target_names=labels)
print(report)

model.save('ResNet50_DEL.h5')

# Prikazivanje predikcija za nekoliko slika
num_images_to_display = 2

# Dobivanje predikcija za testni skup
Y_pred = model.predict(X_valid)
Y_pred_classes = np.argmax(Y_pred, axis=1)

# Prikazivanje slika s pripadajućim stvarnim i predviđenim klasama
for i in range(num_images_to_display):
    # Indeks slike koju ćemo prikazati
    idx = np.random.randint(0, len(X_valid))
    
    # Prikazivanje slike
    plt.imshow(X_valid[idx])
    plt.title(f"Stvarna klasa: {labels[Y_valid[idx]]}, Predviđena klasa: {labels[Y_pred_classes[idx]]}")
    plt.show()
